# Hussaini lab data (Axona) conversion to NWB

Here, I am going to test code for doing said conversions.

### NWB

To convert to NWB we will need a separate Interface class (e.g. AxonaRecordingExtractorInterface) for each data type / extractor. Those are then combined with an NWBConverter class for the Lab (e.g. HussainiLabNWBConverter), which exports to NWB. 

Note that for Milestone2, we might not need any of this, since spikeextractors has its own NWBconverters for extractor and sorting classes!

Adapted from https://github.com/catalystneuro/movshon-lab-to-nwb/blob/main/tutorials/blackrock_nwb_conversion_detailed.ipynb

In [115]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [116]:
import os

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [117]:
!ls ../nwb-conversion-tools/nwb_conversion_tools -l

total 76
-rwxrwxrwx 1 steburg steburg    69 Apr  5 11:28 __init__.py
drwxrwxrwx 1 steburg steburg  4096 Apr 12 09:49 __pycache__
-rwxrwxrwx 1 steburg steburg  1404 Apr  5 11:28 auto_qc.py
-rwxrwxrwx 1 steburg steburg  1338 Apr  5 11:28 basedatainterface.py
-rwxrwxrwx 1 steburg steburg  2134 Apr  5 11:28 baseimagingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2775 Apr  5 11:28 baselfpextractorinterface.py
-rwxrwxrwx 1 steburg steburg  5127 Apr 12 09:46 baserecordingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2363 Apr  5 11:28 basesegmentationextractorinterface.py
-rwxrwxrwx 1 steburg steburg  3831 Apr 12 09:46 basesortingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  5003 Apr  8 17:38 conversion_tools.py
drwxrwxrwx 1 steburg steburg  4096 Apr 12 09:46 datainterfaces
-rwxrwxrwx 1 steburg steburg  5099 Apr 12 09:46 json_schema_utils.py
-rwxrwxrwx 1 steburg steburg 10154 Apr  5 11:28 metafile.schema.json
-rwxrwxrwx 1 steburg steburg  6665 Apr  5 11:28 nwbconverter.py
dr

In [118]:
# Import modules

import random
import string
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile
import numpy as np
import re
import datetime

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface,  BaseDataInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from nwb_conversion_tools.json_schema_utils import get_schema_from_method_signature, get_base_schema, fill_defaults
from nwb_conversion_tools.datainterfaces.interface_utils.brpylib import NsxFile
from nwb_conversion_tools import SpikeGLXRecordingInterface

In [119]:
# from basedatainterface.py
base_schema = get_base_schema(
    id_='metadata.schema.json',
    root=True,
    title='Metadata',
    description='Schema for the metadata',
    version="0.1.0",
)

In [120]:
base_schema

{'required': [],
 'properties': {},
 'type': 'object',
 'additionalProperties': True,
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'metadata.schema.json',
 'title': 'Metadata',
 'description': 'Schema for the metadata',
 'version': '0.1.0'}

In [121]:
glx = SpikeGLXRecordingInterface

In [122]:
glx.get_source_schema()

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [123]:
import inspect

inspect.signature(glx.__init__).parameters

mappingproxy({'self': <Parameter "self">,
              'file_path': <Parameter "file_path: Union[str, pathlib.Path, NoneType]">,
              'stub_test': <Parameter "stub_test: Union[bool, NoneType] = False">})

In [124]:
!ls ../spikeextractors/spikeextractors -l

total 224
-rwxrwxrwx 1 steburg steburg  1265 Mar 19 08:48 __init__.py
drwxrwxrwx 1 steburg steburg  4096 Apr 12 09:47 __pycache__
-rwxrwxrwx 1 steburg steburg 22834 Mar 19 08:52 baseextractor.py
-rwxrwxrwx 1 steburg steburg  9269 Mar 19 08:52 cacheextractors.py
drwxrwxrwx 1 steburg steburg  4096 Apr  9 15:25 example_datasets
-rwxrwxrwx 1 steburg steburg   105 Mar 19 08:48 exceptions.py
-rwxrwxrwx 1 steburg steburg 36890 Mar 19 08:52 extraction_tools.py
-rwxrwxrwx 1 steburg steburg  5952 Apr 10 17:04 extractorlist.py
drwxrwxrwx 1 steburg steburg  4096 Mar 22 09:47 extractors
-rwxrwxrwx 1 steburg steburg  6273 Apr  9 15:25 multirecordingchannelextractor.py
-rwxrwxrwx 1 steburg steburg  8558 Apr  9 15:25 multirecordingtimeextractor.py
-rwxrwxrwx 1 steburg steburg  5458 Mar 19 08:52 multisortingextractor.py
-rwxrwxrwx 1 steburg steburg 41357 Mar 19 08:52 recordingextractor.py
-rwxrwxrwx 1 steburg steburg 30642 Mar 19 08:52 sortingextractor.py
-rwxrwxrwx 1 steburg steburg  8382 Apr  9 15:25

In [125]:
from spikeextractors.extractors.neoextractors import AxonaRecordingExtractor

In [126]:
are = AxonaRecordingExtractor(filename=filename)
print('Number of channels:', are.get_num_channels())
print('Channel groups:', are.get_channel_groups())

Number of channels: 16
Channel groups: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Parse .set file for metadata to include in metadata.

In [127]:
def parse_generic_header(filename, params):
    """
    Given a binary file with phrases and line breaks, enters the
    first word of a phrase as dictionary key and the following
    string (without linebreaks) as value. Returns the dictionary.
    
    INPUT
    filename (str): .set file path and name.
    params (list or set): parameter names to search for. 
    
    OUTPUT
    header (dict): dictionary with keys being the parameters that
                   were found & values being strings of the data.
                   
    EXAMPLE
    parse_generic_header('myset_file.set', ['experimenter', 'trial_time'])
    """
    header = {}
    params = set(params)
    with open(filename, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                break
            line = bin_line.decode('cp1252').replace('\r\n', '').replace('\r', '').strip()
            parts = line.split(' ')
            key = parts[0]
            if key in params:
                header[key] = ' '.join(parts[1:])
            
    return header

In [128]:
params_of_interest = [
    'experimenter', 
    'comments',
    'duration', 
    'sw_version',
    'tracker_version',
    'stim_version',
    'audio_version'
]

In [129]:
parse_generic_header(set_file, params_of_interest)

NameError: name 'set_file' is not defined

In [ ]:
def read_iso_datetime(set_file):
    """ 
    Creates datetime object (y, m, d, h, m, s) from .set file header 
    """
    with open(set_file, 'r', encoding='cp1252') as f:
        for line in f:
            if line.startswith('trial_date'):
                date_string = re.findall(r'\d+\s\w+\s\d{4}$', line)[0]
            if line.startswith('trial_time'):
                time_string = line[len('trial_time')+1::].replace('\n', '')

    return datetime.datetime.strptime(date_string + ', ' + time_string, \
        "%d %b %Y, %H:%M:%S").isoformat()

In [ ]:
class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = {
            'required': ['filename'],
            'properties': {
                'filename': {
                    'type': 'string',
                    'format': 'file',
                    'description': 'Path to Axona files.'
                }
            },
            'type': 'object',
            'additionalProperties': True
        }
        return source_schema
        
    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""        
                
        # Extract information for specific parameters from .set file
        params_of_interest = [
            'experimenter', 
            'comments',
            'duration', 
            'sw_version',
            'tracker_version',
            'stim_version',
            'audio_version'
        ]
        set_file = self.source_data['filename']+'.set'
        par = parse_generic_header(set_file, params_of_interest)
        
        # Extract information from AxonaRecordingExtractor
        elec_group_names = self.recording_extractor.get_channel_groups()
        unique_elec_group_names = set(elec_group_names)
        
        # Add available metadata
        metadata = super().get_metadata()
        metadata['NWBFile'] = dict(
            session_start_time=read_iso_datetime(set_file),
            session_description=par['comments'],
            session_duration=par['duration']+'s',
            experimenter=[par['experimenter']]
        )
        
        metadata['Ecephys'] = dict(
            Device=[
                dict(
                    name="Axona",
                    description="Axona DacqUSB, sw_version={}".format(par['sw_version']),
                    manufacturer="Axona"
                ),
            ],
            ElectrodeGroup=[
                dict(
                    name=f'Group{group_name}',
                    location='',
                    device='Axona',
                    description=f"Group {group_name} electrodes.",
                )
                for group_name in unique_elec_group_names
            ],
            Electrodes=[
                dict(
                    name='group_name',
                    description="The name of the ElectrodeGroup this electrode is a part of.",
                    data=[f"Group{x}" for x in elec_group_names]
                )
            ],
            ElectricalSeries=dict(
                name='ElectricalSeries',
                description="Raw acquisition traces."
            )
        )
  
        return metadata

In [ ]:
# TEST CODE IN ONE CELL

import os
import json
from jsonschema import validate
import spikeextractors as se
import re
import datetime

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)


def read_iso_datetime(set_file):
    """ 
    Creates datetime object (y, m, d, h, m, s) from .set file header 
    """
    with open(set_file, 'r', encoding='cp1252') as f:
        for line in f:
            if line.startswith('trial_date'):
                date_string = re.findall(r'\d+\s\w+\s\d{4}$', line)[0]
            if line.startswith('trial_time'):
                time_string = line[len('trial_time')+1::].replace('\n', '')

    return datetime.datetime.strptime(date_string + ', ' + time_string, \
        "%d %b %Y, %H:%M:%S").isoformat()


class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = {
            'required': ['filename'],
            'properties': {
                'filename': {
                    'type': 'string',
                    'format': 'file',
                    'description': 'Path to Axona files.'
                }
            },
            'type': 'object',
            'additionalProperties': True
        }
        return source_schema
        
    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""        
                
        # Extract information for specific parameters from .set file
        set_file = self.source_data['filename']+'.set'
        
        # Add available metadata
        metadata = super().get_metadata()
        metadata['NWBFile'] = dict(
            session_start_time=read_iso_datetime(set_file),
        )
        
        metadata['Ecephys'] = dict(
            Device=[
                dict(
                    name='Axona',
                    description='Device'
                )
            ],
            ElectrodeGroup=[
                dict(
                    name=f'shank{n + 1}',
                    description=f"shank{n + 1} electrodes",
                    location='',
                    device='Axona'
                )
                for n, _ in enumerate([1])
            ],
            ElectricalSeries=dict(
                name='ElectricalSeries'
            )
        )
  
        return metadata


# Define source data (required for instantiating converter)
source_data = dict(
    AxonaRecordingExtractorInterface=dict(
        filename=filename
    )
)
print(json.dumps(source_data, indent=2))


# Create a Hussaini-lab converter
from nwb_conversion_tools import NWBConverter

class HussainiRawNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface
    )

    
# Instantiate Hussaini-lab converter
converter = HussainiRawNWBConverter(source_data=source_data)


# View metadata_schema from converter
metadata_schema = converter.get_metadata_schema()
print(json.dumps(metadata_schema['properties'], indent=2))


# View converter metadata
metadata = converter.get_metadata()
print(json.dumps(metadata, indent=2))


# Validate metadata with metadata_schema
validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

In [ ]:
RXI = AxonaRecordingExtractorInterface(filename=filename)

In [ ]:
RXI.get_metadata()

In [ ]:
base_metadata_schema = RXI.get_metadata_schema()

In [ ]:
import json
print(json.dumps(base_metadata_schema, indent=2))

In [ ]:
# The metadata_schema does not validate with the metadata, bc
# the full metadata_schema is only inherited when defining the
# nwbconverter (see below).

from jsonschema import validate, ValidationError

try:
    validate(
        instance=RXI.get_metadata(),
        schema=RXI.get_metadata_schema()
    )
    print('validation SUCCESS')
except ValidationError:
    print('validation FAILED')

In [130]:
# So let's create an AxonaNWBconverter

# Here we will also import all necessary datainterfaces
from nwb_conversion_tools import (
    NWBConverter
)


# A separate class for each datatype
class HussainiRawNWBConverter(NWBConverter):
    
    '''
    # If I want to be able to add arbitrary additional / optional properties
    def get_metadata_schema(cls):
        metadata_schema = super().get_metadata_schema()
        metadata_schema['properties']['NWBFile']['additionalProperties'] = True
        return metadata_schema
    '''
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface, 
        # AxonaSortingExtractorInterface=AxonaSortingExtractorInterface
    )

In [131]:
# Get source_schema from converter

source_schema = HussainiRawNWBConverter.get_source_schema()
print(json.dumps(source_schema['properties'], indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "required": [
      "filename"
    ],
    "properties": {
      "filename": {
        "type": "string",
        "format": "file",
        "description": "Path to Axona files."
      }
    },
    "type": "object",
    "additionalProperties": true
  }
}


In [132]:
# Define source data (required for instantiating converter)

source_data = dict(
    AxonaRecordingExtractorInterface=dict(
        filename=filename
    )
)

print(json.dumps(source_data, indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s"
  }
}


In [133]:
# Instantiate Hussaini-lab converter

converter = HussainiRawNWBConverter(source_data=source_data)

In [134]:
# Get metadata_schema from converter

metadata_schema = converter.get_metadata_schema()
print(json.dumps(metadata_schema['properties'], indent=2))

{
  "NWBFile": {
    "required": [
      "session_description",
      "identifier",
      "session_start_time"
    ],
    "properties": {
      "session_description": {
        "type": "string",
        "format": "long",
        "description": "a description of the session where this data was generated",
        "default": ""
      },
      "identifier": {
        "type": "string",
        "description": "a unique text identifier for the file",
        "default": "a0697ae0-e8ba-4820-ac9f-114581fac68d"
      },
      "session_start_time": {
        "type": "string",
        "description": "the start date and time of the recording session",
        "format": "date-time",
        "default": "2020-10-04T11:07:07"
      },
      "experimenter": {
        "type": "array",
        "items": {
          "type": "string",
          "title": "experimenter"
        },
        "description": "name of person who performed experiment",
        "default": [
          "Abid"
        ]
      },
      "e

In [135]:
metadata = converter.get_metadata()
metadata

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': 'cac6c794-5f4c-48d0-9847-d84875321b9a',
  'session_duration': '600.00625s',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this                         electrode is a part of.',
    'data': ['Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0']}],
  'ElectricalSeries': {'name': 'ElectricalSeries',
   'description': 'Raw acquisition traces.'}}}

In [136]:
metadata['Ecephys']

{'Device': [{'name': 'Axona',
   'description': 'Axona DacqUSB, sw_version=1.2.2.16',
   'manufacturer': 'Axona'}],
 'ElectrodeGroup': [{'name': 'Group0',
   'location': '',
   'device': 'Axona',
   'description': 'Group 0 electrodes.'}],
 'Electrodes': [{'name': 'group_name',
   'description': 'The name of the ElectrodeGroup this                         electrode is a part of.',
   'data': ['Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0',
    'Group0']}],
 'ElectricalSeries': {'name': 'ElectricalSeries',
  'description': 'Raw acquisition traces.'}}

In [137]:
# Validate metadata with metadata_schema from nwbconverter

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

## Test axonadatainterface from nwb_conversion_tools

In [138]:
import json
from jsonschema import validate

In [139]:
import os

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [140]:
#from nwb_conversion_tools import axonadatainterface

In [141]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface
)

In [142]:
import sys
sys.path.append('/mnt/d/spikeinterface/hussaini-lab-to-nwb/hussaini_lab_to_nwb')

In [143]:
from hussaininwbconverter import HussainiNWBConverter

In [144]:
# Get source_schema from converter

source_schema = HussainiNWBConverter.get_source_schema()
print(json.dumps(source_schema['properties'], indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "required": [
      "filename"
    ],
    "properties": {
      "filename": {
        "type": "string",
        "format": "file",
        "description": "Path to Axona files."
      }
    },
    "type": "object",
    "additionalProperties": true
  }
}


In [145]:
source_data = dict(
    AxonaRecordingExtractorInterface=dict(
        filename=filename
    )
)
print(json.dumps(source_data, indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s"
  }
}


In [146]:
# Validate source_data with source_schema

validate(
    instance=source_data,
    schema=source_schema
)

In [147]:
# Instantiate Hussaini-lab converter

converter = HussainiNWBConverter(source_data=source_data)

In [148]:
# Get metadata_schema from converter

metadata_schema = converter.get_metadata_schema()
print(json.dumps(metadata_schema['properties'], indent=2))


metadata = converter.get_metadata()
metadata

{
  "NWBFile": {
    "required": [
      "session_description",
      "identifier",
      "session_start_time"
    ],
    "properties": {
      "session_description": {
        "type": "string",
        "format": "long",
        "description": "a description of the session where this data was generated",
        "default": ""
      },
      "identifier": {
        "type": "string",
        "description": "a unique text identifier for the file",
        "default": "f815f467-6dd1-43fa-b1f4-7b1e61e9b8c5"
      },
      "session_start_time": {
        "type": "string",
        "description": "the start date and time of the recording session",
        "format": "date-time",
        "default": "2020-10-04T11:07:07"
      },
      "experimenter": {
        "type": "array",
        "items": {
          "type": "string",
          "title": "experimenter"
        },
        "description": "name of person who performed experiment",
        "default": [
          "Abid"
        ]
      },
      "e

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': 'bff4a978-7d4d-4878-9c3b-c4c9a0a28e02',
  'session_duration': '600.00625s',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this                         electrode is a part of.',
    'data': ['Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0']}],
  'ElectricalSeries': {'name': 'ElectricalSeries',
   'description': 'Raw acquisition traces.'}}}

In [149]:
# Validate metadata with metadata_schema from nwbconverter

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

In [185]:
print(json.dumps(converter.get_metadata_schema(), indent=2))

{
  "required": [
    "NWBFile"
  ],
  "properties": {
    "NWBFile": {
      "required": [
        "session_description",
        "identifier",
        "session_start_time"
      ],
      "properties": {
        "session_description": {
          "type": "string",
          "format": "long",
          "description": "a description of the session where this data was generated",
          "default": ""
        },
        "identifier": {
          "type": "string",
          "description": "a unique text identifier for the file",
          "default": "e7dcc951-ef88-4dcb-a2fe-0a79b258133c"
        },
        "session_start_time": {
          "type": "string",
          "description": "the start date and time of the recording session",
          "format": "date-time",
          "default": "2020-10-04T11:07:07"
        },
        "experimenter": {
          "type": "array",
          "items": {
            "type": "string",
            "title": "experimenter"
          },
          "descrip

In [150]:
# Test pytest function test_interface_schemas() for this specific interface

from jsonschema import Draft7Validator

In [151]:
data_interface = AxonaRecordingExtractorInterface(filename=filename)

schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)

In [40]:
data_interface.convert_data?

In [41]:
data_interface.run_conversion?

In [152]:
dir_name

'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'

In [43]:
HussainiNWBConverter.run_conversion?

In [153]:
converter.run_conversion(metadata=metadata, nwbfile_path=os.path.join(dir_name, 'axonadatainterface_test.nwb'))

UnsupportedOperation: Cannot read data from file /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axonadatainterface_test.nwb in mode 'r+'. There are no values.

In [154]:
from nwb_conversion_tools.conversion_tools import get_default_nwbfile_metadata
from nwb_conversion_tools.json_schema_utils import dict_deep_update


In [155]:
metadata = dict_deep_update(get_default_nwbfile_metadata(), metadata)

In [156]:
metadata['NWBFile']

{'session_description': '',
 'session_start_time': '2020-10-04T11:07:07',
 'identifier': 'bff4a978-7d4d-4878-9c3b-c4c9a0a28e02',
 'session_duration': '600.00625s',
 'experimenter': ['Abid']}

In [157]:
metadata_test = metadata
metadata_test['NWBFile'].pop('session_duration') 

'600.00625s'

In [158]:
metadata_test

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': 'bff4a978-7d4d-4878-9c3b-c4c9a0a28e02',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this                         electrode is a part of.',
    'data': ['Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0']}],
  'ElectricalSeries': {'name': 'ElectricalSeries',
   'description': 'Raw acquisition traces.'}}}

In [159]:
converter.data_interface_objects

{'AxonaRecordingExtractorInterface': <nwb_conversion_tools.datainterfaces.axonadatainterface.AxonaRecordingExtractorInterface at 0x7fa2ad3957f0>}

In [160]:
metadata["NWBFile"]

{'session_description': '',
 'session_start_time': '2020-10-04T11:07:07',
 'identifier': 'bff4a978-7d4d-4878-9c3b-c4c9a0a28e02',
 'experimenter': ['Abid']}

In [161]:
from datetime import datetime
from pynwb import NWBFile

In [162]:
def make_nwbfile_from_metadata(metadata: dict):
    """Make NWBFile from available metadata."""
    metadata = dict_deep_update(get_default_nwbfile_metadata(), metadata)
    nwbfile_kwargs = metadata["NWBFile"]
    if "Subject" in metadata:
        # convert ISO 8601 string to datetime
        if "date_of_birth" in metadata["Subject"] and isinstance(metadata["Subject"]["date_of_birth"], str):
            metadata["Subject"]["date_of_birth"] = datetime.fromisoformat(metadata["Subject"]["date_of_birth"])
        nwbfile_kwargs.update(subject=Subject(**metadata["Subject"]))
    # convert ISO 8601 string to datetime
    if isinstance(nwbfile_kwargs.get("session_start_time", None), str):
        nwbfile_kwargs["session_start_time"] = datetime.fromisoformat(metadata["NWBFile"]["session_start_time"])
    return NWBFile(**nwbfile_kwargs)

In [163]:
nwbfile = make_nwbfile_from_metadata(metadata=metadata_test)

In [164]:
print(type(metadata_test['Ecephys']['Device']))
print(type(metadata_test['Ecephys']['Device'][0]))

dict

In [165]:
data_interface

In [166]:
nwbfile

root pynwb.file.NWBFile at 0x140334994408064
Fields:
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 4, 12, 15, 13, 11, 872443, tzinfo=tzlocal())]
  identifier: bff4a978-7d4d-4878-9c3b-c4c9a0a28e02
  session_start_time: 2020-10-04 11:07:07-04:00
  timestamps_reference_time: 2020-10-04 11:07:07-04:00

In [167]:
import spikeextractors as se
import numpy as np

In [168]:
r_cache = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_data_preproc.pkl'))

Versions are not the same. This might lead to errors. Use  spikeextractors version 0.9.5


In [104]:
n_channels = max([len(x['data']) for x in metadata['Ecephys']['Electrodes']])

recording = se.NumpyRecordingExtractor(timeseries=np.array(range(n_channels)), sampling_frequency=1)
se.NwbRecordingExtractor.add_devices(
    recording=recording,
    nwbfile=nwbfile,
    metadata=metadata
)

In [169]:
nwbfile

root pynwb.file.NWBFile at 0x140334994408064
Fields:
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 4, 12, 15, 13, 11, 872443, tzinfo=tzlocal())]
  identifier: bff4a978-7d4d-4878-9c3b-c4c9a0a28e02
  session_start_time: 2020-10-04 11:07:07-04:00
  timestamps_reference_time: 2020-10-04 11:07:07-04:00

In [103]:
se.NwbRecordingExtractor.write_recording(
    recording=r_cache,
    nwbfile=nwbfile,
    metadata=metadata
)

AssertionError: Expected metadata['Ecephys']['Device'] to be a list of dictionaries!

In [109]:
data_interface.run_conversion?

In [107]:
data_interface.run_conversion(nwbfile, metadata)

AssertionError: Expected metadata['Ecephys']['Device'] to be a list of dictionaries!

In [108]:
converter.run_conversion(metadata=metadata_test,
                         save_to_file=False,
                         nwbfile_path=None,
                         nwbfile=None)

AssertionError: Expected metadata['Ecephys']['Device'] to be a list of dictionaries!

In [170]:
se.NwbRecordingExtractor.add_devices(
    recording=recording,
    nwbfile=nwbfile,
    metadata=metadata
)

In [171]:
metadata['Ecephys']['Device']

[{'name': 'Axona',
  'description': 'Axona DacqUSB, sw_version=1.2.2.16',
  'manufacturer': 'Axona'}]

In [176]:
if nwbfile is not None:
    assert isinstance(nwbfile, NWBFile), "'nwbfile' should be of type pynwb.NWBFile"
if len(nwbfile.devices) == 0:
    se.NwbRecordingExtractor.add_devices(recording, nwbfile)
defaults = dict(
    name="Electrode Group",
    description="no description",
    location="unknown",
    device_name="Device"
)
if metadata is None or 'ElectrodeGroup' not in metadata['Ecephys']:
    metadata = dict(
        Ecephys=dict(
            ElectrodeGroup=[defaults]
        )
    )

In [181]:
assert all([isinstance(x, dict) for x in metadata['Ecephys']['ElectrodeGroup']]), \
    "Expected metadata['Ecephys']['ElectrodeGroup'] to be a list of dictionaries!"

In [183]:
for grp in metadata['Ecephys']['ElectrodeGroup']:
    device_name = grp.get('device_name', defaults['device_name'])
    print(device_name)

Device


In [184]:
metadata['Ecephys']['ElectrodeGroup']

[{'name': 'Group0',
  'location': '',
  'device': 'Axona',
  'description': 'Group 0 electrodes.'}]

In [182]:
for grp in metadata['Ecephys']['ElectrodeGroup']:
    device_name = grp.get('device_name', defaults['device_name'])
    if grp.get('name', defaults['name']) not in nwbfile.electrode_groups:
        if device_name not in nwbfile.devices:
            new_device = dict(
                Ecephys=dict(
                    Device=dict(
                        name=device_name
                    )
                )
            )
            se.NwbRecordingExtractor.add_devices(recording, nwbfile, metadata=new_device)
            warnings.warn(f"Device \'{device_name}\' not detected in "
                          "attempted link to electrode group! Automatically generating.")
        electrode_group_kwargs = dict(defaults, **grp)
        electrode_group_kwargs.pop('device_name')
        electrode_group_kwargs.update(device=nwbfile.devices[device_name])
        nwbfile.create_electrode_group(**electrode_group_kwargs)

AssertionError: Expected metadata['Ecephys']['Device'] to be a list of dictionaries!

In [175]:
len(nwbfile.devices)

1

In [172]:
se.NwbRecordingExtractor.add_electrode_groups(
    recording=recording,
    nwbfile=nwbfile,
    metadata=metadata
)

AssertionError: Expected metadata['Ecephys']['Device'] to be a list of dictionaries!

In [ ]:
se.NwbRecordingExtractor.add_electrodes(
    recording=recording,
    nwbfile=nwbfile,
    metadata=metadata,
    write_scaled=write_scaled
)

In [30]:
converter.run_conversion(metadata=metadata_test,
                         save_to_file=False,
                         nwbfile_path=os.path.join(dir_name, 'axonadatainterface_test.nwb'),
                         nwbfile=None)

AttributeError: 'str' object has no attribute 'get'

In [18]:
import nwb_conversion_tools

In [19]:
# Try to see for which data_interface pytest crashes
# Only for tutorialinterfaces! But it works fine for AxonaRecordingExtractorInterface!

interfaces_that_crash = [nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialRecordingInterface,
                         nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialSortingInterface]

from nwb_conversion_tools import interface_list

for data_interface in interface_list:
    
    if not data_interface in interfaces_that_crash:
    
        # check validity of source schema
        schema = data_interface.get_source_schema()
        Draft7Validator.check_schema(schema)

        # check validity of conversion options schema
        schema = data_interface.get_conversion_options_schema()
        Draft7Validator.check_schema(schema)

In [20]:
interfaces_that_crash = [nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialRecordingInterface,
                         nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialSortingInterface]

data_interface = interfaces_that_crash[0]
        
# check validity of source schema
schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)

In [21]:
data_interface = interfaces_that_crash[1]
        
# check validity of source schema
schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)